In [3]:
from pathlib import Path

In [4]:
data_dir = Path().resolve().parent / "data"
data_dir

WindowsPath('F:/Dokumente/Uni/Master/Zweites Semester (Master)/Natural Language Processing/Projekt/modern-talking/data')

In [48]:
# data analysis and manipulation tool 
import pandas
# natural language token analyzer package
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\maxhe\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.


True

In [15]:
df_arguments_dev = pandas.read_csv(data_dir / "arguments_dev.csv")
df_arguments_dev

,arg_id,argument,topic,stance
0,arg_4_0,having a school uniform can reduce bullying as...,We should abandon the use of school uniform,-1
1,arg_4_1,it is a good way to instill discipline,We should abandon the use of school uniform,-1
2,arg_4_2,it is cheaper for parents to buy school unifor...,We should abandon the use of school uniform,-1
3,arg_4_3,one of the benefits of school uniforms is that...,We should abandon the use of school uniform,-1
4,arg_4_4,researchers found that in schools across the w...,We should abandon the use of school uniform,-1
...,...,...,...,...
927,arg_15_222,we should end it and give people that are the ...,We should end affirmative action,1
928,arg_15_223,we should end it so that every persons are equal,We should end affirmative action,1
929,arg_15_224,we should evaluate candidates based solely on ...,We should end affirmative action,1
930,arg_15_225,we should select people based on their merit o...,We should end affirmative action,1


In [80]:
df_arguments_dev['argument_tokens'] = df_arguments_dev.apply(lambda row: nltk.word_tokenize(row['argument']), axis=1)
df_arguments_dev


,arg_id,argument,topic,stance,argument_tokens
0,arg_4_0,having a school uniform can reduce bullying as...,We should abandon the use of school uniform,-1,"[having, a, school, uniform, can, reduce, bull..."
1,arg_4_1,it is a good way to instill discipline,We should abandon the use of school uniform,-1,"[it, is, a, good, way, to, instill, discipline]"
2,arg_4_2,it is cheaper for parents to buy school unifor...,We should abandon the use of school uniform,-1,"[it, is, cheaper, for, parents, to, buy, schoo..."
3,arg_4_3,one of the benefits of school uniforms is that...,We should abandon the use of school uniform,-1,"[one, of, the, benefits, of, school, uniforms,..."
4,arg_4_4,researchers found that in schools across the w...,We should abandon the use of school uniform,-1,"[researchers, found, that, in, schools, across..."
...,...,...,...,...,...
927,arg_15_222,we should end it and give people that are the ...,We should end affirmative action,1,"[we, should, end, it, and, give, people, that,..."
928,arg_15_223,we should end it so that every persons are equal,We should end affirmative action,1,"[we, should, end, it, so, that, every, persons..."
929,arg_15_224,we should evaluate candidates based solely on ...,We should end affirmative action,1,"[we, should, evaluate, candidates, based, sole..."
930,arg_15_225,we should select people based on their merit o...,We should end affirmative action,1,"[we, should, select, people, based, on, their,..."


In [16]:
df_arguments_train = pandas.read_csv(data_dir / "arguments_train.csv")
df_arguments_train

,arg_id,argument,topic,stance
0,arg_0_0,`people reach their limit when it comes to the...,Assisted suicide should be a criminal offence,-1
1,arg_0_1,A patient should be able to decide when they h...,Assisted suicide should be a criminal offence,-1
2,arg_0_2,a person has the right to end their suffering ...,Assisted suicide should be a criminal offence,-1
3,arg_0_3,a person should have the dignity to choose how...,Assisted suicide should be a criminal offence,-1
4,arg_0_4,a person should have the right to be able to c...,Assisted suicide should be a criminal offence,-1
...,...,...,...,...
5578,arg_27_218,we should subsidize vocational education to en...,We should subsidize vocational education,1
5579,arg_27_219,We should subsidize vocational education to su...,We should subsidize vocational education,1
5580,arg_27_220,While many who graduate from universities stru...,We should subsidize vocational education,1
5581,arg_27_221,with the rising cost of college tuition vocati...,We should subsidize vocational education,1


In [10]:
df_key_points_dev = pandas.read_csv(data_dir / "key_points_dev.csv")
df_key_points_dev

,key_point_id,key_point,topic,stance
0,kp_4_0,Children can still express themselves using ot...,We should abandon the use of school uniform,-1
1,kp_4_1,School uniform reduces bullying,We should abandon the use of school uniform,-1
2,kp_4_2,School uniforms encourage discipline or focus ...,We should abandon the use of school uniform,-1
3,kp_4_3,School uniforms saves costs,We should abandon the use of school uniform,-1
4,kp_4_4,School uniforms create a sense of equality/unity,We should abandon the use of school uniform,-1
5,kp_4_5,School uniform is harming the student's self e...,We should abandon the use of school uniform,1
6,kp_4_6,School uniforms are expensive,We should abandon the use of school uniform,1
7,kp_4_7,School uniforms are often uncomfortable/sexist,We should abandon the use of school uniform,1
8,kp_4_8,School uniform harms learning/creativity,We should abandon the use of school uniform,1
9,kp_4_9,School uniforms increase conformity or harm in...,We should abandon the use of school uniform,1


In [35]:
df_key_points_train = pandas.read_csv(data_dir / "key_points_train.csv")
df_key_points_train

,key_point_id,key_point,topic,stance
0,kp_0_0,Assisted suicide gives dignity to the person t...,Assisted suicide should be a criminal offence,-1
1,kp_0_1,Assisted suicide reduces suffering,Assisted suicide should be a criminal offence,-1
2,kp_0_2,People should have the freedom to choose to en...,Assisted suicide should be a criminal offence,-1
3,kp_0_3,The terminally ill would benefit from assisted...,Assisted suicide should be a criminal offence,-1
4,kp_0_4,Assisted suicide allows people to solicit some...,Assisted suicide should be a criminal offence,1
...,...,...,...,...
202,kp_27_3,subsidizing vocational education is expensive,We should subsidize vocational education,-1
203,kp_27_4,subsidizing vocational education promotes thos...,We should subsidize vocational education,1
204,kp_27_5,vocational education is a good career choice,We should subsidize vocational education,1
205,kp_27_6,vocational education better fits many students,We should subsidize vocational education,1


In [36]:
# labels_dev = pandas.read_csv(data_dir / "labels_dev.csv")
# labels_dev

In [37]:
# labels_train = pandas.read_csv(data_dir / "labels_train.csv")
# labels_train